In [1]:
# English to Korean Translation with Tensorflow RNN
# written by Sung Kyu Lim
# limsk@ece.gatech.edu
# 1/7/2019


# import related packages
import tensorflow as tf
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'


# 41 characters used in the dictionary
# S, E, and P are special characters used in RNN
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)
n_class = n_input = dic_len


# training data
# the english word should be of length 4, and the korean 2
seq_data = [['word', '단어'], ['wood', '나무'], ['game', '놀이'], ['girl', '소녀'], ['kiss', '키스'], ['love', '사랑']]


# one-hot encoding function
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]

        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)

    return input_batch, output_batch, target_batch


# global parameters
learning_rate = 0.01
n_hidden = 128
total_epoch = 100


# in Seq2Seq RNN, we use the following placeholder type
# for the encoder and decoder:
# [batch size, time steps, input size]
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])


# in Seq2Seq RNN, we use the following placeholder type
# for the output: [batch size, time steps]
targets = tf.placeholder(tf.int64, [None, None])


# encoder cell definition
# we use dropout to avoid overfitting
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    _ , enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)


# decoder cell definition
# we use dropout to avoid overfitting
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

    # in Seq2Seq model, we use the encoder output state
    # as the initial state for the decoder
    dec_outputs, _ = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=enc_states, dtype=tf.float32)


# instead of tf.matmul(outputs, W)+b, 
# we use the 'dense' function in tensorflow layers:
# 'dense' produces [batch_size, time_step, input_size]
# in our case, model shape is (?, ?, 41)
model = tf.layers.dense(dec_outputs, n_class, activation=None)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))
opt = tf.train.AdamOptimizer(learning_rate).minimize(cost)


# print model and target information
def info():
    sample = [['word', '단어']]
    input_batch, output_batch, target_batch = make_batch(sample)
    m_info, t_info = sess.run([model, targets], feed_dict = {enc_input: input_batch, 
        dec_input: output_batch, targets: target_batch})

    print('input:', sample)
    print('model\n', m_info)
    print('target:', t_info)
    print()


# training session
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([opt, cost], feed_dict={enc_input: input_batch, dec_input: output_batch, 
        targets: target_batch})
    print('epoch: %04d' % epoch, 'cost: %.4f' % loss)

info()


# test new words
new_data = [['word', 'PP'], ['wodr', 'PP'], ['love', 'PP'], ['loev', 'PP'], ['abcd', 'PP'], 
    ['wide', 'PP'], ['gate', 'PP']]
input_batch, output_batch, target_batch = make_batch(new_data)
prediction = tf.argmax(model, 2)
result = sess.run(prediction, feed_dict = {enc_input: input_batch, dec_input: output_batch, 
    targets: target_batch})

for i in range(len(new_data)):
    decoded = [char_arr[i] for i in result[i]]
    korean = decoded[0] + decoded[1]
    print('english:', new_data[i][0], ', result:', result[i], ', korean:', korean)


/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


epoch: 0000 cost: 3.7452
epoch: 0001 cost: 2.6045
epoch: 0002 cost: 1.4894
epoch: 0003 cost: 1.3045
epoch: 0004 cost: 0.7632
epoch: 0005 cost: 0.5393
epoch: 0006 cost: 0.6049
epoch: 0007 cost: 0.3859
epoch: 0008 cost: 0.2063
epoch: 0009 cost: 0.3731
epoch: 0010 cost: 0.1874
epoch: 0011 cost: 0.1958
epoch: 0012 cost: 0.2331
epoch: 0013 cost: 0.0930
epoch: 0014 cost: 0.1765
epoch: 0015 cost: 0.0428
epoch: 0016 cost: 0.0727
epoch: 0017 cost: 0.1663
epoch: 0018 cost: 0.0993
epoch: 0019 cost: 0.0732
epoch: 0020 cost: 0.0862
epoch: 0021 cost: 0.0622
epoch: 0022 cost: 0.0131
epoch: 0023 cost: 0.0185
epoch: 0024 cost: 0.0076
epoch: 0025 cost: 0.0196
epoch: 0026 cost: 0.0484
epoch: 0027 cost: 0.0245
epoch: 0028 cost: 0.0074
epoch: 0029 cost: 0.0040
epoch: 0030 cost: 0.0052
epoch: 0031 cost: 0.0076
epoch: 0032 cost: 0.0043
epoch: 0033 cost: 0.0188
epoch: 0034 cost: 0.0073
epoch: 0035 cost: 0.0016
epoch: 0036 cost: 0.0022
epoch: 0037 cost: 0.0031
epoch: 0038 cost: 0.0038
epoch: 0039 cost: 0.0111
